#Cargar liberias, leer archivos, verificar columnas

In [36]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import missingno as msno
import numpy as np

In [37]:
train = pd.read_csv('train.csv')
train.shape

(1460, 81)

In [38]:
train = train.drop('Id',axis=1)
train.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [39]:
#train.columns

In [40]:
#train.select_dtypes('number').info()

In [41]:
num = train.select_dtypes(include=['number'])
categ = train.select_dtypes(include=['object'])
numeric_cols = train.select_dtypes(include=['number']).columns
categ_cols = train.select_dtypes(include=['object']).columns

Exploración de los datos

Caracterización de las variables numericas

Eliminación de valores faltantes

In [42]:
categ_null=categ.isnull().sum().sort_values(ascending=False)
categ[categ_null[categ_null > 0].index].isnull().sum().sort_values(ascending=False)
categ_null = categ_null[categ_null >0]
#Todas las columnas categoricas con valores faltantes los pueden reemplazar con 'None', indican que no tienen no estan incompletos los datos

In [43]:
train[['LotFrontage','MasVnrArea']] = train[['LotFrontage','MasVnrArea']].fillna(0)
train.dropna(subset=['GarageYrBlt'],inplace=True)
##GarageYrBlt se podrían borrar solo son 5.5%, o pasar a 0.
train[categ_null.index] = train[categ_null.index].fillna('None')

In [44]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000


#Guardar df sin preprocesamiento y creación de json

In [45]:
train_clean = train #Backup del df ya limpio pero sin procesar
train_clean.to_csv('train_clean.csv', index=False)


In [46]:
no_numbers = {'1stFlrSF':'FirstFlrSF','2ndFlrSF':'SecondFlrSF','3SsnPorch':'ThreeSsnPorch'}
train_clean.rename(columns=no_numbers,inplace=True)

In [47]:
train_cleanjson = train_clean.copy()

In [48]:
json_train= train_clean.to_json(orient='records',indent=5)
with open('houses.json', 'w') as json_file:
    json_file.write(json_train)

In [49]:
var_map = {'Id':'Id',
 'MSSubClass':'Tipo de vivienda en venta',
 'MSZoning':'Zonificacion',
 'LotFrontage': 'Pies conectados a la propiedad',
 'LotArea': 'Tamaño del lote',
 'Street': 'Calle',
 'Alley': 'Pasillo',
 'LotShape': 'Forma',
 'LandContour': 'Planitud',
 'Utilities': 'Utilidades',
 'LotConfig': 'Configuracion del lote',
 'LandSlope':'Pendiente',
 'Neighborhood':'Colonia',
 'Condition1':'Proximidad a condiciones',
 'Condition2':'Proximidad a otras condiciones',
 'BldgType':'Tipo de vivienda',
 'HouseStyle':'Estilo',
 'OverallQual': 'Calidad',
 'OverallCond':'Condicion',
 'YearBuilt':'Año de construccion',
 'YearRemodAdd':'Año de remodelacion',
 'RoofStyle':'Estilo de techo',
 'RoofMatl':'Material del techo',
 'Exterior1st':'Revestimiento exterior',
 'Exterior2nd':'Otro revestimiento exterior',
 'MasVnrType':'Canteria',
 'MasVnrArea':'Area de canteria',
 'ExterQual':'Calidad de materiales exteriores',
 'ExterCond':'Condicion de materiales exteriores',
 'Foundation':'Cimentacion',
 'BsmtQual':'Altura de sotano',
 'BsmtCond':'Estado del sotano',
 'BsmtExposure':'Paredes de planta baja/jardin',
 'BsmtFinType1':'Califacion de superficie del sotano',
 'BsmtFinSF1':'Acabados tipo 1',
 'BsmtFinType2':'Otra califiacion de la superficie acabada del sotano',
 'BsmtFinSF2':'Acabados tipo 2',
 'BsmtUnfSF':'Superficie del sotano sin terminar',
 'TotalBsmtSF':'Superficie total del sotano',
 'Heating':'Tipo de calefaccion',
 'HeatingQC':'Calidad de calefaccion',
 'CentralAir':'Aire acondicionado central',
 'Electrical':'Sistema electrico',
 '1stFlrSF':'Primer piso',
 '2ndFlrSF':'Segundo piso',
 'LowQualFinSF':'Acabados de baja calidad',
 'GrLivArea':'Superficie habitable arriba del suelo',
 'BsmtFullBath':'Baños completos en el sotano',
 'BsmtHalfBath':'Medios baños en el sotano',
 'FullBath':'Baños completos arriba del suelo',
 'HalfBath':'Medios baños arriba del suelo',
 'BedroomAbvGr':'Recamaras arriba del suelo',
 'KitchenAbvGr':'Cocina arriba del suelo',
 'KitchenQual':'Calidad de cocina',
 'TotRmsAbvGrd':'Total de habitaciones arriba del suelo',
 'Functional':'Funcionalidad',
 'Fireplaces':'Chimeneas',
 'FireplaceQu': 'Calidad de chimenea',
 'GarageType':'Tipo de garage',
 'GarageYrBlt':'Año de construccion de garage',
 'GarageFinish':'Terminacion del garage',
 'GarageCars':'Capacidad de coches en el garage',
 'GarageArea':'Area del garage',
 'GarageQual':'Calidad de garage',
 'GarageCond':'Condicion del garage',
 'PavedDrive':'Calzada pavimentada',
 'WoodDeckSF':'Superficie de la cubierta',
 'OpenPorchSF':'Area de porche',
 'EnclosedPorch':'Area de porche cerrado',
 '3SsnPorch':'Area de porche de tres estaciones',
 'ScreenPorch':'Area del porche cubierto',
 'PoolArea':'Area de la piscina',
 'PoolQC':'Calidad de la piscina',
 'Fence':'Calidad de valla',
 'MiscFeature':'Elementos extra',
 'MiscVal':'Precio de elemento extra',
 'MoSold':'Mes de venta',
 'YrSold':'Año de venta',
 'SaleType':'Tipo de venta',
 'SaleCondition':'Condicion de venta',
 'SalePrice':'Precio de venta'}

In [50]:
#train_cleanjson.rename(columns=var_map,inplace=True)

In [51]:
train_cleanjson.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,FirstFlrSF,SecondFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000


In [52]:
train_cleanjson.shape

(1379, 80)

In [53]:

import json
casas_dict = train_cleanjson.to_dict(orient='records')
casas_json = json.dumps({"casas": casas_dict}, indent=4)
with open('casas.json', 'w') as archivo:
    archivo.write(casas_json)

In [54]:
casas_dict[0]

{'MSSubClass': 60,
 'MSZoning': 'RL',
 'LotFrontage': 65.0,
 'LotArea': 8450,
 'Street': 'Pave',
 'Alley': 'None',
 'LotShape': 'Reg',
 'LandContour': 'Lvl',
 'Utilities': 'AllPub',
 'LotConfig': 'Inside',
 'LandSlope': 'Gtl',
 'Neighborhood': 'CollgCr',
 'Condition1': 'Norm',
 'Condition2': 'Norm',
 'BldgType': '1Fam',
 'HouseStyle': '2Story',
 'OverallQual': 7,
 'OverallCond': 5,
 'YearBuilt': 2003,
 'YearRemodAdd': 2003,
 'RoofStyle': 'Gable',
 'RoofMatl': 'CompShg',
 'Exterior1st': 'VinylSd',
 'Exterior2nd': 'VinylSd',
 'MasVnrType': 'BrkFace',
 'MasVnrArea': 196.0,
 'ExterQual': 'Gd',
 'ExterCond': 'TA',
 'Foundation': 'PConc',
 'BsmtQual': 'Gd',
 'BsmtCond': 'TA',
 'BsmtExposure': 'No',
 'BsmtFinType1': 'GLQ',
 'BsmtFinSF1': 706,
 'BsmtFinType2': 'Unf',
 'BsmtFinSF2': 0,
 'BsmtUnfSF': 150,
 'TotalBsmtSF': 856,
 'Heating': 'GasA',
 'HeatingQC': 'Ex',
 'CentralAir': 'Y',
 'Electrical': 'SBrkr',
 'FirstFlrSF': 856,
 'SecondFlrSF': 854,
 'LowQualFinSF': 0,
 'GrLivArea': 1710,
 'BsmtF

#Preprocesamiento de datos

##Creación de dummy variables

In [55]:
'''
dummies = pd.get_dummies(train[categ.columns])
# Combina las dummy variables con el DataFrame original
train = pd.concat([train, dummies], axis=1)
# Elimina la columna original
train = train.drop(categ.columns, axis=1)
'''

'\ndummies = pd.get_dummies(train[categ.columns])\n# Combina las dummy variables con el DataFrame original\ntrain = pd.concat([train, dummies], axis=1)\n# Elimina la columna original\ntrain = train.drop(categ.columns, axis=1)\n'

In [56]:
train.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,FirstFlrSF,SecondFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500


In [57]:
train.shape

(1379, 80)

In [58]:
num.shape[1] + categ.shape[1]

80

In [59]:
#train.corr().sort_values(by='SalePrice',ascending=False)['SalePrice'].head(15)

#Modelos

In [60]:
#train_clean.columns[train_clean.isnull().sum()>0]

In [61]:
X = train_clean.drop('SalePrice',axis=1)
y = train_clean['SalePrice']

In [62]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,FirstFlrSF,SecondFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal


In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [64]:
#PREPROCESAMIENTO
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
#Escalamiento para las variables numericas
#Dummies para las variables categoricas
preprocessing = ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_cols[:-1]),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categ_cols)
    ])

In [65]:
from sklearn.pipeline import Pipeline

from sklearn.ensemble import GradientBoostingRegressor
modelo = GradientBoostingRegressor(learning_rate=0.2,n_estimators=200) #0.018
pipeline = Pipeline(steps=[('preprocessor', preprocessing),
                           ('model', modelo)])



In [66]:
#X_train.columns

In [68]:
pipeline.fit(X, y)
#y_pred = pipeline.predict(X)

ValueError: A given column is not a column of the dataframe

In [ ]:
#Evaluar modelo
from sklearn.metrics import mean_squared_error
import math
math.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))

0.04632793120591926

In [ ]:
pipeline.predict(X_test)

array([147325.03970589, 226890.91966059, 125374.427557  , 170426.3477501 ,
       540148.78567895, 186976.97543061, 142045.74854375, 191365.96373082,
        84555.99382493, 169566.01728557, 104362.74468594, 175831.89595334,
       179434.89064096, 272669.13914773, 181224.32861059, 142549.21861082,
       159576.82272776, 108863.92361983, 143037.00671331, 152903.99350763,
       308373.36379765, 103573.66013693, 160359.0265593 , 392105.77818647,
       199649.78553985, 220943.22010454, 184665.8113511 ,  89272.0946202 ,
       152498.07485615, 236916.34776377, 319863.26153944, 163320.63882932,
       173674.9071128 , 159899.8803632 , 146381.04212959, 133440.57939622,
       141769.02759025, 132095.17198784, 119581.86470626, 152047.53527361,
       175851.24639683, 139241.96537532, 236414.54597472, 472624.12591466,
       194307.91203088, 110228.89331433,  69890.98886565, 193565.71575066,
       163039.75509216, 197299.12048167, 197201.59127733, 436654.78003598,
       186848.87385219, 1

In [ ]:
X_test.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

#Test

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test.isna().sum()

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       227
LotArea             0
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType        894
MasVnrArea         15
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           44
BsmtCond           45
BsmtExposure       44
BsmtFinType1       42
BsmtFinSF1          1
BsmtFinType2       42
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
Heating             0
HeatingQC           0
CentralAir          0
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [ ]:
X_ = test.drop(['Id'],axis=1)
y_ = test['Id']
media = X_[['YearBuilt', 'YearRemodAdd','GarageYrBlt','YrSold']].mean()
X_[['YearBuilt', 'YearRemodAdd','GarageYrBlt','YrSold']].fillna(media)
X_.fillna(0,inplace=True)
results = pipeline.predict(X_)

In [ ]:
y_['SalePrice'] = results

<ipython-input-328-75eca8f8178d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_['SalePrice'] = results


In [ ]:
csv =pd.DataFrame({'Id':test['Id'], 'SalePrice': results})

In [ ]:
csv.to_csv('prediction_gbc_lr0_2_ne250_alldata.csv',index=False)

In [ ]:
csv.head()

,Id,SalePrice
0,1461,124606.988889
1,1462,174082.012928
2,1463,187819.407037
3,1464,189834.297541
4,1465,186039.276176


In [ ]:
train_clean.to_csv('train_clean.csv')

In [ ]:
train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.00000
mean,56.022480,57.766497,10695.812183,6.187092,5.577955,1972.958666,1985.435098,108.364757,455.578680,48.102248,570.765047,1074.445975,1177.129804,353.424946,4.134880,1534.689630,0.430747,0.058738,1.580131,0.395939,2.865120,1.038434,6.552574,0.641769,1978.506164,1.870921,500.762146,97.456853,47.276287,21.039159,3.609862,15.945613,2.920957,42.889050,6.334300,2007.812183,185479.51124
std,41.026897,35.038221,10214.702133,1.345780,1.081031,29.379883,20.444852,184.195220,459.691379,164.324665,443.677845,436.371874,387.014961,439.553171,40.130546,519.140514,0.514052,0.238285,0.548051,0.502402,0.783961,0.196048,1.589821,0.645985,24.689725,0.630055,185.680520,126.699192,65.210465,60.535107,30.154682,57.249593,41.335545,501.613931,2.696674,1.330221,79023.89060
min,20.000000,0.000000,1300.000000,2.000000,2.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,438.000000,0.000000,0.000000,438.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,1900.000000,1.000000,160.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,35311.00000
25%,20.000000,41.500000,7741.000000,5.000000,5.000000,1955.000000,1968.000000,0.000000,0.000000,0.000000,228.000000,810.000000,894.000000,0.000000,0.000000,1154.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,380.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,134000.00000
50%,50.000000,64.000000,9591.000000,6.000000,5.000000,1976.000000,1994.000000,0.000000,400.000000,0.000000,476.000000,1008.000000,1098.000000,0.000000,0.000000,1479.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,484.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,167500.00000
75%,70.000000,79.000000,11708.500000,7.000000,6.000000,2001.000000,2004.000000,170.500000,732.000000,0.000000,811.000000,1314.000000,1414.000000,738.500000,0.000000,1790.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,580.000000,171.000000,69.500000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,217750.00000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,2.000000,2.000000,3.000000,2.000000,6.000000,3.000000,12.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.00000
